# Vor BioPharm Technical Assessment

## Will Richards

### 1. Data Download from AWS S3

First I set up an Anaconda environment to work in using Python 3.7.

Then, I downloaded `s3://vor-interview/sampleinfo/datatable.tsv` using the AWS CLI tool after some configuration.

Note: The `aws configure list` and file copying can be done in Jupyter by leading with an exclamation, `!`.

In [ ]:
from IPython.display import Image
Image(filename='Screen Shot 2021-07-13 at 6.13.11 PM.png')

In [ ]:
Image(filename='Screen Shot 2021-07-13 at 8.07.34 PM.png') 

In [ ]:
Image(filename='Screen Shot 2021-07-13 at 6.13.54 PM.png')

In [ ]:
# Review contents of file to ensure proper download.
import pandas as pd

inputfile = "~/Desktop/vorBioTest/datatable.tsv"

rawDataDF = pd.read_csv(inputfile, delimiter="\t")
print(rawDataDF.head(3))

In [ ]:
print(rawDataDF.tail(3))

In [ ]:
print(rawDataDF.info())

In [ ]:
print(rawDataDF.describe())

In [ ]:
print(rawDataDF.columns)

### 2. Import the data table to a portable SQLite database

In [ ]:
import sqlite3

dbFile = "vorBioTest.db"
conn = sqlite3.connect(dbFile)
c = conn.cursor()
c.execute("""CREATE TABLE EXPERIMENTS (SPPTleID text,
                                       FilePath text,
                                       FileSize number,
                                       experiment_name text,
                                       experiment_id text,
                                       vendor text,
                                       target_gene text,
                                       owner text,
                                       date text,
                                       email text)""")
conn.commit()

In [ ]:
rawDataDF.to_sql("EXPERIMENTS",
                 conn,
                 if_exists="replace",
                 index=False)

In [ ]:
# Confirm successful upload to DB
c.execute("SELECT * FROM EXPERIMENTS LIMIT 10")

for row in c.fetchall():
    print(row)

### 3.  Use your R or Python for data analysis and visualization

I chose to work in a Jupyter Notebook as much as possible. Use 'Run All Cells' to use the connector from above.

In [ ]:
# Using the established connection from above.
c.execute("SELECT vendor, FileSize FROM EXPERIMENTS")
df = pd.DataFrame(c.fetchall(), columns=["vendor", "fileSize"])

Using the Pandas `describe` tool and `groupyby` `median()` function to retrieve summary statistics by vendor.

In [ ]:
stats_by_vendor = {}
for v in df.groupby("vendor"):
    print(v[0])
    print(v[1].describe())
    print(v[1].median()[0])

Using the same connection and `plotly` I created `bar` chats by vendors for montly data file size.

In [ ]:
c.execute("SELECT vendor, date, FileSize FROM EXPERIMENTS")
plotting_df = pd.DataFrame(c.fetchall(), columns=["vendor", "date", "fileSize"])

mos = []
dys = []
yrs = []
for d in plotting_df.date.tolist():
    mm, dd, yr = d.split("/")
    mos.append(mm)
    yrs.append(yr)
    
plotting_df["months"] = mos
plotting_df["years"] = yrs
plotting_df["moYrs"] = plotting_df["months"] + "/" + plotting_df["years"]

vendor_month_sum = []
for v in plotting_df.groupby(["vendor", "moYrs"]):
    vendor_month_sum.append([v[0][0], v[0][1], sum(v[1]["fileSize"])])

df_to_plot = pd.DataFrame(vendor_month_sum, columns = ['vendor', 'monthYear', 'totalFileSize'])
df_to_plot["monthYear"] =  pd.to_datetime(df_to_plot["monthYear"], format="%m/%y")
df_to_plot = df_to_plot.sort_values(["monthYear"]).copy()

In [ ]:
import plotly.express as px

fig = px.bar(df_to_plot, x="monthYear", y="totalFileSize", color="vendor", text="totalFileSize")
fig.update_layout(title_text="Monthly Data File Size by Vendor")
fig.update_xaxes(title="Month - Year")
fig.update_yaxes(title="Total File Size")
fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.show()


### 4. Deploy a web server for the SQLite database query using docker container 'coleifer/sqlite-web:latest'

A. Access docker image: https://github.com/coleifer/sqlite-web

B. Save the deploy steps as a markdown documentation with a few screenshots.

In [ ]:
# While I was able to access the GitHub repository, I do not have a Docker account and
# experienced what may be a bug due to this.
Image(filename="Screen Shot 2021-07-14 at 8.09.07 AM.png")

However, entering the command to launch to database's graphical user interface ("GUI") worked.
> sqlite_web vorBioTest.db

In [ ]:
Image(filename="Screen Shot 2021-07-14 at 8.08.45 AM.png")